# NLP - Homework 1
### Miguel Bonilla

 [Compare Name](#1.-Compare-Name)

1. Compare your given name with your nickname (if you don’t have a nickname, invent one for this 
assignment) by answering the following questions:
a. What is the edit distance between your nickname and your given name?
b. What is the percentage string match between your nickname and your given name?
Show your work for both calculations.
2. Find a friend (or family member or classmate) who you know has read a certain book. Without 
your friend knowing, copy the first two sentences of that book. Now rewrite the words from 
those sentences, excluding stop words. Now tell your friend to guess which book the words are 
from by reading them just that list of words. Did you friend correctly guess the book on the first 
try? What did he or she guess? Explain why you think you friend either was or was not able to 
guess the book from hearing the list of words. 
3. Run one of the stemmers available in Python. Run the same two sentences from question 2 
above through the stemmer and show the results. How many of the outputted stems are valid 
morphological roots of the corresponding words? Express this answer as a percentage.

In [31]:
import copy
import numpy as np
import pandas as pd
import nltk
import contractions

### 1. Compare Name
#### Part a: Edit Distance

Given Name: Miguel - Nickname: Migue 

Setting up the matrix manually:

-|M|i|g|u|e|l
-|-|-|-|-|-|-|
M|0|1|2|3|4|5
i|1|0|1|2|3|4
g|2|1|0|1|2|3
u|3|2|1|0|1|2
e|4|3|2|1|0|1

Distance between Miguel and Migue = 1. Since they are the same with one fewer letter for the nickname.

In [17]:
## Calculating distance using formula from book:
def levenshtein_edit_distance(u, v):
    # convert to lower case
    u = u.lower()
    v = v.lower()
    # base cases
    if u == v: return 0
    elif len(u) == 0: return len(v)
    elif len(v) == 0: return len(u)
    # initialize edit distance matrix
    edit_matrix = []
    # initialize two distance matrices
    du = [0] * (len(v) + 1)
    dv = [0] * (len(v) + 1)
    # du: the previous row of edit distances
    for i in range(len(du)):
        du[i] = i
    # dv : the current row of edit distances
    for i in range(len(u)):
        dv[0] = i + 1
        # compute cost as per algorithm
        for j in range(len(v)):
            cost = 0 if u[i] == v[j] else 1
            dv[j + 1] = min(dv[j] + 1, du[j + 1] + 1, du[j] + cost)
        # assign dv to du for next iteration
        for j in range(len(du)):
            du[j] = dv[j]
        # copy dv to the edit matrix
        edit_matrix.append(copy.copy(dv))
    # compute the final edit distance and edit matrix
    distance = dv[len(v)]
    edit_matrix = np.array(edit_matrix)
    edit_matrix = edit_matrix.T
    edit_matrix = edit_matrix[1:,]
    edit_matrix = pd.DataFrame(data=edit_matrix,
                               index=list(v),
                               columns=list(u))
    print("distance = ",distance)
    return edit_matrix

In [13]:
levenshtein_edit_distance('Miguel','Migue')

distance =  1


,m,i,g,u,e,l
m,0,1,2,3,4,5
i,1,0,1,2,3,4
g,2,1,0,1,2,3
u,3,2,1,0,1,2
e,4,3,2,1,0,1


#### Part b: String Match

In [60]:
### from Harry Potter and The Sorcerer's Stone
text_string = '''Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal,
thank you very much. They were the last people you'd expect to be involved in anything strange or mysterious,
because they just didn't hold with such nonsense.'''

In [61]:
### expanding contractions (dev.to/edualgo/handle-contractions-in-text-preprocessing-nlp-21p)
expanded_words=[]
for word in text_string.split():
    expanded_words.append(contractions.fix(word))

In [62]:
stopwords = nltk.corpus.stopwords.words('english')

In [63]:
content = [w for w in expanded_words if w.lower() not in stopwords]

In [64]:
content

['Mr.',
 'Mrs.',
 'Dursley,',
 'number',
 'four,',
 'Privet',
 'Drive,',
 'proud',
 'say',
 'perfectly',
 'normal,',
 'thank',
 'much.',
 'last',
 'people',
 'you would',
 'expect',
 'involved',
 'anything',
 'strange',
 'mysterious,',
 'did not',
 'hold',
 'nonsense.']